In [115]:
import mne
import numpy as np

In [116]:
# Defining frequency bands
delta = (1, 4)
theta = (4, 8)
alpha = (8, 12)
beta = (12, 30)
gamma = (30, 100)
bands = {"delta": delta, "theta": theta, "alpha": alpha, "beta": beta, "gamma": gamma}


In [118]:
#creating a function which would return rest and task PSD values for delta, theta, alpha, beta and gamma
def calculate_psd(subject_id):
  rest_path = f"eeg-during-mental-arithmetic-tasks-1.0.0\{subject_id}_1.edf"
  task_path = f"eeg-during-mental-arithmetic-tasks-1.0.0\{subject_id}_2.edf"

  # Loading EEG data
  rest_raw = mne.io.read_raw_edf(rest_path)
  task_raw = mne.io.read_raw_edf(task_path)

  # Geting data and sampling frequency
  rest_data = rest_raw.get_data()
  task_data = task_raw.get_data()
  rest_sfreq = rest_raw.info['sfreq']
  task_sfreq = task_raw.info['sfreq']

  rest_psd_list = []
  for epoch in rest_data:
    psd, _ = mne.time_frequency.psd_array_multitaper(epoch, rest_sfreq, fmin=1, fmax=100)
    rest_psd_list.append(psd)

  task_psd_list = []
  for epoch in task_data:
    psd, _ = mne.time_frequency.psd_array_multitaper(epoch, task_sfreq, fmin=1, fmax=100)
    task_psd_list.append(psd)

  # Calculating average band power across epochs
  band_power = {}
  for band_name, (fmin, fmax) in bands.items():
    band_power[band_name] = []
    rest=0
    task=0
    for rest_psd, task_psd in zip(rest_psd_list, task_psd_list):
      epoch_delta_psd = np.mean(rest_psd, axis=0)
      rest+=(np.mean(epoch_delta_psd))

      epoch_delta_psd = np.mean(task_psd, axis=0)
      task+=(np.mean(epoch_delta_psd))

    band_power[band_name].append(rest/21)
    band_power[band_name].append(task/21)
  
  return band_power


<string>:3: SyntaxWarning: invalid escape sequence '\{'
<string>:4: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\{'
<string>:3: SyntaxWarning: invalid escape sequence '\{'
<string>:4: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\{'
C:\Users\KIIT\AppData\Local\Temp\ipykernel_21904\1661549804.py:3: SyntaxWarning: invalid escape sequence '\{'
  rest_path = f"eeg-during-mental-arithmetic-tasks-1.0.0\{subject_id}_1.edf"
C:\Users\KIIT\AppData\Local\Temp\ipykernel_21904\1661549804.py:4: SyntaxWarning: invalid escape sequence '\{'
  task_path = f"eeg-during-mental-arithmetic-tasks-1.0.0\{subject_id}_2.edf"


In [119]:
import pandas as pd

In [120]:
#reading the csv file
df=pd.read_csv("eeg-during-mental-arithmetic-tasks-1.0.0\subject-info.csv")
df.head()

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\KIIT\AppData\Local\Temp\ipykernel_21904\2126531006.py:2: SyntaxWarning: invalid escape sequence '\s'
  df=pd.read_csv("eeg-during-mental-arithmetic-tasks-1.0.0\subject-info.csv")


,Subject,Age,Gender,Recording year,Number of subtractions,Count quality
0,Subject00,21,F,2011,9.70,0
1,Subject01,18,F,2011,29.35,1
2,Subject02,19,F,2012,12.88,1
3,Subject03,17,F,2010,31.00,1
4,Subject04,17,F,2010,8.60,0


In [121]:
#adding the collected data from EEG files to the csv file

new_df = pd.DataFrame(columns=['rest_delta', 'task_delta', 'rest_theta', 'task_theta', 'rest_alpha', 'task_alpha', 'rest_beta', 'task_beta', 'rest_gamma', 'task_gamma'])
for index, row in df.iterrows():
    subject = row['Subject']
    psd_data = calculate_psd(subject)
    new_row = {'rest_delta': psd_data['delta'][0], 'task_delta': psd_data['delta'][1],
               'rest_theta': psd_data['theta'][0], 'task_theta': psd_data['theta'][1],
               'rest_alpha': psd_data['alpha'][0], 'task_alpha': psd_data['alpha'][1],
               'rest_beta': psd_data['beta'][0], 'task_beta': psd_data['beta'][1],
               'rest_gamma': psd_data['gamma'][0], 'task_gamma': psd_data['gamma'][1]}

    
    new_row_df = pd.Series(new_row)  
    new_df = pd.concat([new_df, new_row_df.to_frame().T], ignore_index=True)

df = pd.concat([df, new_df], axis=1) 
print(df.head())


Extracting EDF parameters from d:\Documents\Projects\Deep Learning\eeg-during-mental-arithmetic-tasks-1.0.0\Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from d:\Documents\Projects\Deep Learning\eeg-during-mental-arithmetic-tasks-1.0.0\Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum e

C:\Users\KIIT\AppData\Local\Temp\ipykernel_21904\1689601860.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, new_row_df.to_frame().T], ignore_index=True)


    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spe

In [123]:
df.head()

,Subject,Age,Gender,Recording year,Number of subtractions,Count quality,rest_delta,task_delta,rest_theta,task_theta,rest_alpha,task_alpha,rest_beta,task_beta,rest_gamma,task_gamma
0,Subject00,21,F,2011,9.70,0,4.323483e-09,3.614840e-09,4.323483e-09,3.614840e-09,4.323483e-09,3.614840e-09,4.323483e-09,3.614840e-09,4.323483e-09,3.614840e-09
1,Subject01,18,F,2011,29.35,1,1.894558e-09,2.246069e-09,1.894558e-09,2.246069e-09,1.894558e-09,2.246069e-09,1.894558e-09,2.246069e-09,1.894558e-09,2.246069e-09
2,Subject02,19,F,2012,12.88,1,6.210383e-10,2.530553e-09,6.210383e-10,2.530553e-09,6.210383e-10,2.530553e-09,6.210383e-10,2.530553e-09,6.210383e-10,2.530553e-09
3,Subject03,17,F,2010,31.00,1,1.614755e-09,1.385005e-09,1.614755e-09,1.385005e-09,1.614755e-09,1.385005e-09,1.614755e-09,1.385005e-09,1.614755e-09,1.385005e-09
4,Subject04,17,F,2010,8.60,0,2.337862e-09,2.062338e-09,2.337862e-09,2.062338e-09,2.337862e-09,2.062338e-09,2.337862e-09,2.062338e-09,2.337862e-09,2.062338e-09


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Subject                 36 non-null     object 
 1   Age                     36 non-null     int64  
 2   Gender                  36 non-null     object 
 3   Recording year          36 non-null     int64  
 4   Number of subtractions  36 non-null     float64
 5   Count quality           36 non-null     int64  
 6   rest_delta              36 non-null     float64
 7   task_delta              36 non-null     float64
 8   rest_theta              36 non-null     float64
 9   task_theta              36 non-null     float64
 10  rest_alpha              36 non-null     float64
 11  task_alpha              36 non-null     float64
 12  rest_beta               36 non-null     float64
 13  task_beta               36 non-null     float64
 14  rest_gamma              36 non-null     floa

In [125]:
df = pd.concat([df, pd.get_dummies(df['Gender'], dtype='int')], axis=1)

In [126]:
df = df.drop(['Subject','Gender'], axis=1)
df.shape

(36, 16)

In [127]:
df.corr()

,Age,Recording year,Number of subtractions,Count quality,rest_delta,task_delta,rest_theta,task_theta,rest_alpha,task_alpha,rest_beta,task_beta,rest_gamma,task_gamma,F,M
Age,1.000000,0.303482,0.007376,-0.043485,0.096795,0.224352,0.096795,0.224352,0.096795,0.224352,0.096795,0.224352,0.096795,0.224352,-0.382336,0.382336
Recording year,0.303482,1.000000,0.049381,0.165405,-0.158995,0.071610,-0.158995,0.071610,-0.158995,0.071610,-0.158995,0.071610,-0.158995,0.071610,-0.364698,0.364698
Number of subtractions,0.007376,0.049381,1.000000,0.739224,-0.121152,-0.015699,-0.121152,-0.015699,-0.121152,-0.015699,-0.121152,-0.015699,-0.121152,-0.015699,0.010133,-0.010133
Count quality,-0.043485,0.165405,0.739224,1.000000,-0.102710,-0.102485,-0.102710,-0.102485,-0.102710,-0.102485,-0.102710,-0.102485,-0.102710,-0.102485,0.071611,-0.071611
rest_delta,0.096795,-0.158995,-0.121152,-0.102710,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,0.053821,-0.053821
task_delta,0.224352,0.071610,-0.015699,-0.102485,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,-0.068004,0.068004
rest_theta,0.096795,-0.158995,-0.121152,-0.102710,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,0.053821,-0.053821
task_theta,0.224352,0.071610,-0.015699,-0.102485,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,-0.068004,0.068004
rest_alpha,0.096795,-0.158995,-0.121152,-0.102710,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,0.053821,-0.053821
task_alpha,0.224352,0.071610,-0.015699,-0.102485,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,0.821328,1.000000,-0.068004,0.068004


In [35]:
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers


In [128]:
X=df.drop(['Count quality'], axis=1)
y=df['Count quality']

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=104,  test_size=0.25,  shuffle=True) 

In [130]:
#MODEL 1 - Multi-Layer Perceptron (MLP)
from tensorflow import keras


model1 = keras.Sequential([
  keras.layers.Flatten(input_shape=(X_train.shape[1],)),  # Flatten input data
  keras.layers.Dense(128, activation="relu"),  # First hidden layer with 128 neurons and ReLU activation
  keras.layers.Dense(64, activation="relu"),  # Second hidden layer with 64 neurons and ReLU activation
  keras.layers.Dense(1, activation="sigmoid"),  # Output layer with 1 neuron and sigmoid activation
])

# Compile the model
model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model1.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


c:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 900ms/step - accuracy: 0.7037 - loss: 84.2451
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7037 - loss: 62.8971
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7037 - loss: 41.5796
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7037 - loss: 20.2776
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2963 - loss: 3.8740
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2963 - loss: 10.2235
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7037 - loss: 1.3103
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7037 - loss: 3.5042
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7037 - loss: 2.8460
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2963 - loss: 1.8246
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7037 - loss: 1.3469
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6296 - loss: 1.

In [131]:
y_pred1=model1.predict(X_test)
y_pred_binary = np.where(y_pred1[:, 0] > 0.4, 1, 0)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_binary)
# Print the confusion matrix
print(cm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[[0 2]
 [0 7]]


In [133]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#calulating accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
# Calculating precision
precision = precision_score(y_test, y_pred_binary)
# Calculating recall
recall = recall_score(y_test, y_pred_binary)
# Calculating F1-score
f1 = f1_score(y_test, y_pred_binary)

# Printing the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.7777777777777778
Precision: 0.7777777777777778
Recall: 1.0
F1-score: 0.875


In [132]:
#MODEL 2 - xgboost
from xgboost import XGBClassifier
model2 = XGBClassifier(objective="binary:logistic")

# Training the model
model2.fit(X_train, y_train)

# Make predictions on new data
y_pred2 = model2.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred2)

# Print the confusion matrix
print(cm)

[[2 0]
 [0 7]]


In [134]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#calulating accuracy
accuracy = accuracy_score(y_test, y_pred2)
# Calculating precision
precision = precision_score(y_test, y_pred2)
# Calculating recall
recall = recall_score(y_test, y_pred2)
# Calculating F1-score
f1 = f1_score(y_test, y_pred2)

# Printing the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
